In [48]:
# Running on Python 2.7.16 64-bit
import pandas as pd
import numpy as np

In [49]:
summary_df = pd.read_csv(r"summary.csv")
summary_df["WaferOutput"] = summary_df["WaferOutput"].astype(int)
summary_df["TotalSumOfMachines"] = summary_df["TotalSumOfMachines"].astype(int)
summary_df["TotalFootprint"] = summary_df["TotalFootprint"].astype(int)
summary_df["TotalCostOfMachines"] = summary_df["TotalCostOfMachines"].astype(int)
summary_df["DeliveryTime"] = 1.5+(summary_df["TotalSumOfMachines"]/1000)**2+1.2*((summary_df["TotalSumOfMachines"]/1000)**3)
summary_df.head()

,WaferOutput,TotalSumOfMachines,TotalFootprint,TotalCostOfMachines,DeliveryTime
0,0,0,0,0,1.500000
1,100,53,5460,301600000,1.502988
2,200,53,5460,301600000,1.502988
3,300,53,5460,301600000,1.502988
4,400,53,5460,301600000,1.502988


In [50]:
idx_max_footprint = np.nan
idx_max_delivery_time = np.nan

# finding the index for max footprint
for idx in range(summary_df.shape[0]):
    if summary_df["TotalFootprint"][idx] > 50000:
        idx_max_footprint = idx-1
        break

# finding the index for max delivery time
for idx in range(summary_df.shape[0]):
    if summary_df["DeliveryTime"][idx] > 2:
        idx_max_delivery_time = idx-1
        break

In [51]:
def search_summary_df(input_col, input_no, output_col):
    # function: to perform exact linear search and return value from another column but in the same row as the found data
    # return: (int/float) data from the same row as the input data is in 
    # parameters
        # input_col: (str)
        # input_no: (int/float)
        # output_col: (str)
    
    for idx in range(summary_df.shape[0]):
        if summary_df[input_col][idx] == input_no:
            return summary_df[output_col][idx]

In [52]:
def calc_no_of_locations(wafer_output):
    if 0 < wafer_output and wafer_output <= (summary_df["WaferOutput"][idx_max_delivery_time] - summary_df["WaferOutput"][idx_max_footprint]):
        return 1
    else:
        return int(np.ceil((wafer_output-(summary_df["WaferOutput"][idx_max_delivery_time] - summary_df["WaferOutput"][idx_max_footprint]))/float(summary_df["WaferOutput"][idx_max_footprint])))

In [53]:
def calc_sum_of_machines(location_number, wafer_output):
    no_of_locations = calc_no_of_locations(wafer_output)

    # location number 1 which is the location with the existing building needs to considered separately
    if location_number == 1:
        if wafer_output <= summary_df["WaferOutput"][idx_max_delivery_time]:
            return search_summary_df(input_col="WaferOutput", input_no=wafer_output, output_col="TotalSumOfMachines")
        else:
            if wafer_output > summary_df["WaferOutput"][idx_max_footprint]*no_of_locations:
                return search_summary_df(input_col="WaferOutput", input_no=(wafer_output-summary_df["WaferOutput"][idx_max_footprint]*(no_of_locations-1)), output_col="TotalSumOfMachines")
            else:
                return summary_df["TotalSumOfMachines"][idx_max_footprint]
                
    else:
        if wafer_output <= summary_df["WaferOutput"][idx_max_footprint]*(location_number-1)+(summary_df["WaferOutput"][idx_max_delivery_time]-summary_df["WaferOutput"][idx_max_footprint]):
            return 0
        elif wafer_output <= summary_df["WaferOutput"][idx_max_footprint]*(location_number):
            return search_summary_df(input_col="WaferOutput", input_no=(wafer_output-summary_df["WaferOutput"][idx_max_footprint]*(location_number-1)), output_col="TotalSumOfMachines")
        else:
            return summary_df["TotalSumOfMachines"][idx_max_footprint]

In [54]:
def calc_total_CAPEX(sample_df, wafer_output):
    # function: to calculate the CAPEX including with the details of how many machines each location has
    # return: (dataframe) a dataframe 
    
    total_capex = 0
    location_number = 1
    
    new_row ={"WaferOutput": wafer_output, "TotalCAPEX": np.nan, "MachinesAtA": 0, "MachinesAtB": 0, "MachinesAtC": 0, "MachinesAtD": 0, "MachinesAtE": 0}    
    sample_df = sample_df.append(new_row, ignore_index=True)

    while True:
        no_of_machines = calc_sum_of_machines(location_number, wafer_output)
        sample_df[list(sample_df)[location_number+1]][sample_df.shape[0]-1] = no_of_machines

        if no_of_machines == 0:
            break

        # adding cost of machine to CAPEX
        total_capex += search_summary_df(input_col="TotalSumOfMachines", input_no=no_of_machines, output_col="TotalCostOfMachines")

        # adding cost of additional buildings to CAPEX
        if location_number == 1:
            if no_of_machines > summary_df["TotalSumOfMachines"][idx_max_footprint]:
                total_capex += 1070000000
        else:
            if no_of_machines <= summary_df["TotalSumOfMachines"][idx_max_footprint]:
                total_capex += 1000000000
            else:
                total_capex += 1070000000

        location_number += 1
    
    # subtracting the cost of existing machines which we do not need to pay for
    total_capex -= search_summary_df(input_col="WaferOutput", input_no=5000, output_col="TotalCostOfMachines")

    sample_df["TotalCAPEX"][sample_df.shape[0]-1] = total_capex if total_capex > 0 else 0

    return sample_df

In [55]:
answer_df = pd.DataFrame(columns=["WaferOutput", "TotalCAPEX", "MachinesAtA", "MachinesAtB", "MachinesAtC", "MachinesAtD", "MachinesAtE"])
for wafer in range(0, 80000, 100):
    answer_df = calc_total_CAPEX(answer_df, wafer)
    print(wafer)

0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100
6200
6300
6400
6500
6600
6700
6800
6900
7000
7100
7200
7300
7400
7500
7600
7700
7800
7900
8000
8100
8200
8300
8400
8500
8600
8700
8800
8900
9000
9100
9200
9300
9400
9500
9600
9700
9800
9900
10000
10100
10200
10300
10400
10500
10600
10700
10800
10900
11000
11100
11200
11300
11400
11500
11600
11700
11800
11900
12000
12100
12200
12300
12400
12500
12600
12700
12800
12900
13000
13100
13200
13300
13400
13500
13600
13700
13800
13900
14000
14100
14200
14300
14400
14500
14600
14700
14800
14900
15000
15100
15200
15300
15400
15500
15600
15700
15800
15900
16000
16100
16200
16300
16400
16500
16600
16700
16800
16900
17000
17100
17200
17300
17400
17500
17600
17700
17800
17900
18000
18100
18200
18300
18400
18

In [56]:
answer_df.head()

,WaferOutput,TotalCAPEX,MachinesAtA,MachinesAtB,MachinesAtC,MachinesAtD,MachinesAtE
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,100.0,0.0,53.0,0.0,0.0,0.0,0.0
2,200.0,0.0,53.0,0.0,0.0,0.0,0.0
3,300.0,0.0,53.0,0.0,0.0,0.0,0.0
4,400.0,0.0,53.0,0.0,0.0,0.0,0.0


In [57]:
answer_df.to_csv("answer_df_capex.csv", index=False)